Make signal and moment masks for the dwarf galaxies.

Read in the paths from  the `Content` sheet.

In [0]:
!pip install --upgrade --quiet gspread

In [0]:
% pip install --upgrade --quiet astropy
% pip install --quiet spectral_cube
% pip install --quiet reproject
% pip install --quiet scikit-image

In [3]:
# Now install stuff needed for CubeAnalysis
% pip install --quiet sphinx==1.5.6  # avoid issue when creating wheels from repos. https://github.com/sphinx-doc/sphinx/issues/3976
% pip install --quiet FITS_tools image_tools
% pip install --quiet git+https://github.com/radio-astro-tools/uvcombine.git
% pip install --quiet git+https://github.com/e-koch/CubeAnalysis.git

In [4]:
# Make sure this works
from cube_analysis import run_pipeline

import numpy as np

In [5]:
# Mount shared drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [6]:
%cd Shared drives/LocalGroup-VLA
%ls

/gdrive/Shared drives/LocalGroup-VLA
'Code links.gdoc'   feather.ipynb   M31/   mask_and_moments.ipynb   SextansA/
 Content.gsheet     IC1613/         M33/   quicklooks.ipynb         WLM/


Open the Content sheet to get paths that will be run.

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials


gc = gspread.authorize(GoogleCredentials.get_application_default())


In [0]:
import pandas as pd

content_path = '1Dmkioi_BaCdCR-WdPJvqrCwjqQ_aSrIXq0Od9LT8PFg'

worksheet = gc.open_by_key(content_path).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

import pandas as pd
df = pd.DataFrame.from_records(rows)


In [0]:
# df

In [0]:
import string
from datetime import datetime

def update_mask_moments(row, is_feather=False):
    
    # Sign mask and moments in the worksheet with
    # date + time

    if not is_feather:
        col_idx = np.where(colnames == 'Path')[0][0] + 1
        col_idx_mask = np.where(colnames == 'Signal mask')[0][0]
        col_idx_moments = np.where(colnames == 'Moments')[0][0]
    else:
        col_idx = np.where(colnames == 'Feather Path')[0][0] + 1
        col_idx_mask = np.where(colnames == 'Signal mask feather')[0][0]
        col_idx_moments = np.where(colnames == 'Moments feather')[0][0]

    col_letter_mask = string.ascii_uppercase[col_idx_mask]
    col_letter_moments = string.ascii_uppercase[col_idx_moments]

    now = datetime.now()
    
    worksheet.update_acell(f"{col_letter_mask}{row}", now.strftime("%Y-%m-%d %H:%M"))
    worksheet.update_acell(f"{col_letter_moments}{row}", now.strftime("%Y-%m-%d %H:%M"))

# worksheet.update_acell('L2', 'HI!')


In [0]:
# Test worksheet update
# update_mask_moments(2, is_feather=False)

Get the list of VLA-only cubes to run in the pipeline

In [12]:
colnames = np.array(worksheet.row_values('A'))
paths_idx = np.where(colnames == 'Path')[0][0] + 1
runcolab_idx = np.where(colnames == 'Colab_maskmoments')[0][0] + 1
rms_idx = np.where(colnames == '1-sigma Noise (K)')[0][0] + 1
coldens_idx = np.where(colnames == '3-sigma HI column density (cm^-2)')[0][0] + 1

paths = worksheet.col_values(paths_idx)
run_in_colab = worksheet.col_values(runcolab_idx)
paths
# run_in_colab

['Path',
 '',
 'WLM/VLA/13A-213/HI/full_imaging_noSD',
 'WLM/VLA/13A-213/HI/full_imaging_wcont_noSD',
 'WLM/VLA/13A-213/HI/full_imaging_robust0_noSD',
 '',
 'IC1613/VLA/13A-213/HI/full_imaging_noSD',
 'IC1613/VLA/13A-213/HI/full_imaging_wcont_noSD',
 '',
 '',
 'SextansA/VLA/13A-213/HI/full_imaging_noSD',
 'SextansA/VLA/13A-213/HI/full_imaging_CDtaper_noSD',
 'SextansA/VLA/13A-213/HI/full_imaging_Cconfig_noSD',
 '',
 '',
 '',
 'M33/VLA/14B-088/HI/full_imaging_noSD/']

In [0]:
from astropy import log
import astropy.units as u
from astropy.io import fits
from glob import glob
import os
from spectral_cube import SpectralCube
from decimal import Decimal

from cube_analysis import run_pipeline
from cube_analysis.masking import noise_estimation


num_cores = 1

for row, (path, run_colab) in enumerate(zip(paths, run_in_colab)):
    
    if len(path) == "":
        continue

    if run_colab != "T":
        continue
        
    now = datetime.now()
    time_string = now.strftime("%Y-%m-%d-%H-%M")
        
    log_file = os.path.join(path, f'log_mask_and_moments_{time_string}.log')
    with log.log_to_file(log_file):

        log.info(f"Running on cube from: {path}")
        
        cube_name = glob(os.path.join(path, "*image.pbcor.fits"))
        assert len(cube_name) == 1
        cube_name = cube_name[0]
        log.info(f"Found cube name: {cube_name}")
        
        pb_name = glob(os.path.join(path, "*pb.fits"))
        assert len(pb_name) == 1
        pb_name = pb_name[0]
        log.info(f"Found pb name: {pb_name}")
    
        # Convert the cube to K
        cube_K_name = f"{cube_name.rstrip('.fits')}_K.fits"
    
        log.info("Convert cube to K.")
        if not os.path.exists(cube_K_name):
            cube = SpectralCube.read(cube_name, memmap=False)    
            cube.allow_huge_operations = True
    
            cube = cube.to(u.K)
            cube.write(cube_K_name)
            
            del cube
        else:
            log.info("Found existing K cube.")
    
        # Convolve to a common beam size
        cube = SpectralCube.read(cube_K_name)
        if hasattr(cube, 'beams'):
            log.info("Convolving to a common beam size.")
            com_beam = cube.beams.common_beam()
            cube.allow_huge_operations = True
            cube = cube.convolve_to(com_beam)
            # Overwrite K cube
            cube.write(cube_K_name, overwrite=True)
        del cube
    
        # VLA-only cube
        log.info("Masking and moments for the VLA-only cube")
        
        # Seem to have issue creating a FITS file to stream to.
        # Try making an empty file first.
        mask_name = f"{cube_K_name.rstrip('.fits')}_source_mask.fits"
        # touch {mask_name}
        
#         hdulist = fits.HDUList([fits.PrimaryHDU()])
#         hdulist.writeto(mask_name, 'exception')
        
        run_pipeline(cube_K_name,
                     path,
                     pb_file=pb_name,
                     pb_lim=0.15,
                     apply_pbmasking=False,
                     convolve_to_common_beam=False,
                     masking_kwargs={"method": "ppv_connectivity",
                                      "save_cube": True,
                                      "is_huge": False,
                                      "smooth_chans": 5,
                                      "min_chan": 5,
                                      "peak_snr": 4.,
                                      "min_snr": 2,
                                      "edge_thresh": 1,
                                      "pb_map_name": pb_name,
                                     },
                     moment_kwargs={"num_cores": num_cores,
                                    "verbose": True,
                                    "chunk_size": 1e5,
                                    "make_peakvels": True},
                     combeam_kwargs={})


        log.info("Update google sheet w/ write times")
        
        update_mask_moments(row, is_feather=False)
        
        # Calculate the noise in the cube, w/o the signal region
        mask_name = glob(os.path.join(path, "*source_mask.fits"))
        assert len(mask_name) == 1
        mask_name = mask_name[0]
        
        noise_rms, coldens_hi = noise_estimation(cube_K_name, pb_name,
                                                 mask_name)
        
        # Input noise estimation and 3-sigma column density estimation
        # The latter uses the per-channel sensitivity
        log.info("Input noise and column density estimations.")
        col_letter_rms = string.ascii_uppercase[rms_idx]
        col_letter_coldens = string.ascii_uppercase[coldens_idx]
        worksheet.update_acell(f"{col_letter_rms}{row}", '{:.2e}'.format(noise_rms))
        worksheet.update_acell(f"{col_letter_coldens}{row}",
                               '{:.2e}'.format(3 * coldens_hi))
        
        
        print(argh)

INFO: Running on cube from: WLM/VLA/13A-213/HI/full_imaging_noSD [unknown]
INFO: Found cube name: WLM/VLA/13A-213/HI/full_imaging_noSD/WLM_13A-213_HI_spw_0.clean.image.pbcor.fits [unknown]
INFO: Found pb name: WLM/VLA/13A-213/HI/full_imaging_noSD/WLM_13A-213_HI_spw_0.clean.pb.fits [unknown]
INFO: Convert cube to K. [unknown]
INFO: Found existing K cube. [unknown]


/usr/local/lib/python3.6/dist-packages/astropy/convolution/convolve.py:768: RuntimeWarning: invalid value encountered in true_divide
  rifft = (ifftn(fftmult)) / bigimwt


In [0]:

    # VLA+GBT cube
    log.info("Masking and moments for the VLA+EBHIS cube")
    run_pipeline(fourteenA_HI_data_wEBHIS_path("M31_14A_HI_contsub_width_04kms.image.pbcor.EBHIS_feathered.fits"),
                 fourteenA_HI_data_wEBHIS_path("", no_check=True),
                 pb_file=fourteenA_HI_data_path("M31_14A_HI_contsub_width_04kms.pb.fits"),
                 pb_lim=0.05,
                 apply_pbmasking=False,
                 convolve_to_common_beam=False,
                 masking_kwargs={"method": "ppv_connectivity",
                                 "save_cube": True,
                                 "is_huge": True,
                                 "smooth_chans": 17,
                                 "min_chan": 5,
                                 "peak_snr": 4.,
                                 "min_snr": 2,
                                 "edge_thresh": 1,
                                 "pb_map_name": fourteenA_HI_data_path("M31_14A_HI_contsub_width_04kms.pb.fits")
                                 },
                 moment_kwargs={"num_cores": num_cores,
                                "verbose": True,
                                "chunk_size": 1e5,
                                "make_peakvels": False},
                 combeam_kwargs={})